<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Colab%203%20-%20Crear%20una%20Red%20Neuronal%20Artificial%20en%20TensorFlow%202.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-cover.png" />
  Imagen cortesía de: https://www.kaggle.com/
</p>

## Paso 1: Instalar las dependencias y configurar el entorno de GPU

In [1]:
!pip install -U "tensorflow==2.20.0"

## Paso 2: Importar las dependencias necesarias para el proyecto

In [2]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist # pyright: ignore[reportMissingImports]

In [3]:
tf.__version__

'2.20.0'

## Paso 3: Pre procesado de datos



### Cargar el dataset

In [4]:
#Cargar el dataset Fashion Mnist que viene dentro del TensorFlow
# En X_train van a ir cargada 60.000 imagenes
# y_train ss un vector de clase con 10 posibles categorias de clasificacion de cada una de las imagenes
# El X_test cargara las 10.000 imagenes
# y_test es lo mismo pero para las imagenes de X_test
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

### Normalizar las imágenes

Se divide cada imagen en los conjunto de entrenamiento y  de testing entre el valor máximo de cada uno de los píxeles (255).

De este modo, cada píxel se hallará en el rango [0, 1]. Al normalizar las imágenes, nos aseguramos que nuestro modelo de RNA entrenará más rápidamente.

In [5]:
X_train = X_train / 255.0

In [6]:
X_test = X_test / 255.0

### Redimensionar el dataset

Como vamos a utilizar una red neuronal totalmente conectada, vamos a redimensionar los subconjuntos de entrenamiento y testing a formato de vector en lugar de en formato de matriz.

In [7]:
# Al consultar el X_train con shape vemos que tenemos un array tridimencional con 60.000 filas y con 28 * 28 correspondiente a los pixeles
X_train.shape

(60000, 28, 28)

In [8]:
#Como cada imagen tiene 28x28 píxeles, usamos la función reshape en todo el dataset de entrenamiento para convertirlo
# en vectores de tamaño [-1 (todos los elementos), anchura * altura]
X_train = X_train.reshape(-1, 28*28)

In [9]:
# Ahora tenemos un array con 60.000 filas pero los pixeles los aplanamos en vectores/filas para obtener 784 nodos
X_train.shape

(60000, 784)

In [10]:
#Redimensionamos el conjunto de testing del mismo modo
X_test = X_test.reshape(-1, 28*28)

## Paso 4: Construir la Red Neuronal Artificial

###Definir el modelo: Añadir la primera capa totalmente conectada (capa Densa)

Hyper-parametros de la capa:
- número de unidades/neuronas: 128
- función de activación: ReLU
- input_shape: (784, )

In [11]:
# Definir el modelo
# Simplemente se define un objeto de modelo Sequential.
model = tf.keras.Sequential([
    tf.keras.Input(shape=(784,)),
    tf.keras.layers.Dense(128, activation='relu')
])

### Añadir una capa de Dropout

Dropout es una técnica de Regularization donde aleatoriamente se asignan a ciertas neuronas de la red el valor cero. De este modo, mientras se entrena, estas neuronas no actualizarán sus valores. Al tener cierto porcentaje de neuronas sin actualizar, el proceso de entrenamiento toma más tiempo pero por contra tenemos menos posibilidades de sufrir overfitting.

In [12]:
model.add(tf.keras.layers.Dropout(0.2))

### Añadir la segunda capa (capa de salida)

- unidades: número de clases (10 en el caso del Fashion MNIST)
- función de activación: 'softmax' (probabilidades de cada clase)

In [13]:
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Compilar el modelo

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy
- Metricas: Precision

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

### Sumary() nos devueve datos muy interesante:
- Nos indica el Modelo: "sequential"
- Las diferentes capas (Layer)
- Los Parametros de salida (Output Shape)
- Y por ultimo el numero de parametros que la RN debera calcular:
    - 784 X 128 + 128
    - En el caso del Dropout como es aleatorio no tendra que calcular
    - Y las 128 del Dropout X 10 de la capa de salida + 10 adicional

### Entrenar el modelo

In [16]:
# Con el metodo fit podemos entrenar el modelo, pasandole los datos de entrenamiento, sera en 5 epocas y las 60.000 la dividiremos por 32
# verbose=1 → el default. Muestra la barra de progreso con pasos por época (1875/1875),
model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.6805 - sparse_categorical_accuracy: 0.7610
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.4107 - sparse_categorical_accuracy: 0.8528
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.3733 - sparse_categorical_accuracy: 0.8629
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.3436 - sparse_categorical_accuracy: 0.8745
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.3290 - sparse_categorical_accuracy: 0.8773


### Evaluación del modelo y predicción

In [17]:
# Para evaluar un modelo utilizamos evaluate pasandole el conjunto de testing (X_test, y_test)
# evaluate nos devuelve dos elementos, la perdida test_loss y la precicion test_accuracy
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3546 - sparse_categorical_accuracy: 0.8711


In [18]:
print("Test accuracy: {}".format(test_accuracy))

Test accuracy: 0.870199978351593


## Paso 5 : Guardar el modelo

### Guardar la arquitectura (topoligía) de la red neuronal

In [19]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

### Guardar los pesos de la red neuronal

In [20]:
model.save_weights("fashion_model.weights.h5")